<a href="https://colab.research.google.com/github/williamlidberg/Cultural-remains/blob/main/Geographical_Intelligence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training and implementing a machine learning model on geospatial data

Start by doing the iitial setub. Clone the github repository which contains both geospatial data and field data.

In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

!git clone https://github.com/williamlidberg/Geographical-Intelligence.git

The repository include a multiband raster with topographical data and field data collected by Johannes Larsson. Lets start by having a look at the field data

## Field data

In [ ]:
import pandas as pd
soildata = pd.read_csv('/content/Geographical-Intelligence/data/Krycklan_Soilsurvey_data.csv', sep=';')
soildata

In this demo we will use the soil moisture class from the soildata (SMC). Soil moisture was classified into five classes in the field. We will start by inspecting the data.

In [ ]:
!pip install geopandas

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10,20)

gdf = gpd.GeoDataFrame(soildata, geometry=gpd.points_from_xy(soildata.East, soildata.North), crs=3006) # The coordinates are in SWEREFF 99 TM = 3006
gdf.plot(column='SMC', cmap='viridis_r', legend=True)

A geodataframe can be saved as a shapefile which can be handy for later use.

In [ ]:
gdf.to_file('/content/Geographical-Intelligence/data/soildata.shp')  

Investiagte the distribution of our soil moisture classes in Krycklan

In [ ]:
# plot baxplot of SMC

## Rasterdata

In order to train a machine learning model you need some geospatial data.
The following raster layers can be found under '/content/Geographical-Intelligence/data/rasters'

1.   DownslopeIndex_2m
2.   DownslopeIndex_4m
3.   DepthToWater_1ha
4.   DepthToWater_2ha
5.   DepthToWater_4ha
6.   DepthToWater_8ha
7.   DepthToWater_16ha
8.   DepthToWater_32ha
9.   ElevationAboveStream_1ha
10.  ElevationAboveStream_2ha
11.  ElevationAboveStream_4ha
12.  ElevationAboveStream_8ha
13.  ElevationAboveStream_16ha
14.  ElevationAboveStream_32ha
15.  PennocLandformClassification
16.  PlanCurvature
17.  RelativeTopographicPosition
18.  TopographicWetnessIndex
19.  WILT
20.  DEM
21.  Slope
22.  DInfFlowaccumulation

You need to extract the pixel values to the field plots. This can be done using a combination of [rasterio](https://rasterio.readthedocs.io/en/latest/) and [geopandas](https://geopandas.org/en/stable/).

## Extract values to field plots

In [ ]:
!pip install rasterio

In [ ]:
import rasterio
import geopandas as gpd

# Read points from shapefile
soilplots = gpd.read_file('/content/Geographical-Intelligence/data/soildata.shp') # you can use the geodataframe from before but this is to demonstrate how to do it from a shapefile
soilplots.index = range(len(soilplots))
coords = [(x,y) for x, y in zip(soilplots.geometry.x, soilplots.geometry.y)]

# Open the raster using rasterio and extract the pixel values to the geodataframe
# dem
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/dem/16m.tif')
soilplots['dem'] = [x[0] for x in src.sample(coords)] # Naming is important to keep things in order
# Slope
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/Slope/16m.tif')
soilplots['Slope'] = [x[0] for x in src.sample(coords)] 
# PlanCurvature
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/PlanCurvature/16m.tif')
soilplots['PlanCurvature'] = [x[0] for x in src.sample(coords)] 
# RelativeTopographicPosition
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/RelativeTopographicPosition/16m.tif')
soilplots['RelativeTopographicPosition'] = [x[0] for x in src.sample(coords)] 
# PennockLandFormClass
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/PennockLandFormClass/16m.tif')
soilplots['PennockLandFormClass'] = [x[0] for x in src.sample(coords)] 
# TopographicWetnessIndex
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/TopographicWetnessIndex/16m.tif')
soilplots['TopographicWetnessIndex'] = [x[0] for x in src.sample(coords)] 
# WILT
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/WILT/16m.tif')
soilplots['WILT'] = [x[0] for x in src.sample(coords)] 
# DownslopeIndex_2m
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/DownslopeIndex_2m/16m.tif')
soilplots['DownslopeIndex_2m'] = [x[0] for x in src.sample(coords)] 
# DownslopeIndex_4m
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/DownslopeIndex_4m/16m.tif')
soilplots['DownslopeIndex_4m'] = [x[0] for x in src.sample(coords)] 
# DepthToWater_1ha
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/DepthToWater_1ha/16m.tif')
soilplots['DepthToWater_1ha'] = [x[0] for x in src.sample(coords)] 
# DepthToWater_2ha
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/DepthToWater_2ha/16m.tif')
soilplots['DepthToWater_2ha'] = [x[0] for x in src.sample(coords)] 
# DepthToWater_4ha
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/DepthToWater_4ha/16m.tif')
soilplots['DepthToWater_4ha'] = [x[0] for x in src.sample(coords)] 
# DepthToWater_8ha
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/DepthToWater_8ha/16m.tif')
soilplots['DepthToWater_8ha'] = [x[0] for x in src.sample(coords)]
# DepthToWater_16ha
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/DepthToWater_16ha/16m.tif')
soilplots['DepthToWater_16ha'] = [x[0] for x in src.sample(coords)] 
# DepthToWater_32ha
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/DepthToWater_32ha/16m.tif')
soilplots['DepthToWater_32ha'] = [x[0] for x in src.sample(coords)]
# ElevationAboveStream_1ha
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/ElevationAboveStream_1ha/16m.tif')
soilplots['ElevationAboveStream_1ha'] = [x[0] for x in src.sample(coords)]
# ElevationAboveStream_2ha
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/ElevationAboveStream_2ha/16m.tif')
soilplots['ElevationAboveStream_2ha'] = [x[0] for x in src.sample(coords)] 
# ElevationAboveStream_4ha
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/ElevationAboveStream_4ha/16m.tif')
soilplots['ElevationAboveStream_4ha'] = [x[0] for x in src.sample(coords)] 
# ElevationAboveStream_8ha
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/ElevationAboveStream_8ha/16m.tif')
soilplots['ElevationAboveStream_8ha'] = [x[0] for x in src.sample(coords)] 
# ElevationAboveStream_16ha
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/ElevationAboveStream_16ha/16m.tif')
soilplots['ElevationAboveStream_16ha'] = [x[0] for x in src.sample(coords)] 
# ElevationAboveStream_32ha
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/ElevationAboveStream_32ha/16m.tif')
soilplots['ElevationAboveStream_32ha'] = [x[0] for x in src.sample(coords)] 



## Train a random forest model
In this case we will use random forest since it is a robust model that preformes well on most data with minimal tuning. Since we are only interested in soil moisture now we will drop the other Y-variables. We also need to split the data into training data and testing data. The model will be trained on the training data and evaluated on the test data. 

In [ ]:
import pandas as pd

soilplots_dataframe = pd.DataFrame(soilplots.drop(columns='geometry'))
soilplots_dataframe

In [ ]:

soilplots_dataframe.drop(['Humusform', 'SMC', 'HumusformC','OTh_plot','OTh_pit','SoilType','North', 'East'], axis=1, inplace=True)
#soilplots_dataframe.drop(['Humusform', 'SMC','OTh_plot','OTh_pit','SoilType','North', 'East'], axis=1, inplace=True)


Split the data into training data (80%) and testing data (20%). sklearn have a handy module for this purpose.

In [ ]:
from sklearn.model_selection import train_test_split
y = soilplots_dataframe.iloc[:,0] # This is soil moisture
x = soilplots_dataframe.iloc[:,1:] # These are all the topographical variables

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=0, stratify = y)

Now we can define and train (fit) the model. Note that we can choose between a model for [classification](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=randomforest#sklearn.ensemble.RandomForestClassifier) or [regression](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html?highlight=randomforest#sklearn.ensemble.RandomForestRegressor). 

Set up a basic tune grid to try a few different combinations of hyperparameters. More details on what each of the hyperparameters do can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)


In [ ]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
model = RandomForestClassifier() # note that we are using classification for the soil moisture classes


tune_grid = {'n_estimators': [50, 100, 500],
               'max_features': ['sqrt'],
               'max_depth': [4,5,6],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True]}

rf_random = RandomizedSearchCV(estimator = model, param_distributions = tune_grid, random_state=0, n_jobs = -1)

# Train the model using the defined parameters
rf_random.fit(x_train, y_train)
print('The best combination of hyperparameters was', rf_random.best_params_)

Evaluate the model on the test data and inspect which hyperparameters were most useful. First we need to import some more usefull tools from sklearn. Remember that the soil moisture classes were



*   1 = Dry
*   2 = Mesic
*   3 = Mesic-moist
*   4 = Moist
*   2 = Wet


In [ ]:
from sklearn.metrics import classification_report

y_pred_test = rf_random.predict(x_test)
print(classification_report(y_test, y_pred_test, zero_division=0))

## Implement model on raster data
Now we have a working model that we want to apply to the Krycklan catchment. We need to read all the rasterlayers, apply the model and then save the result as a new raster. 

All rasterdata will be read into numpy arrays using gdal.

In [ ]:
from osgeo import gdal_array
import numpy as np
# Read raster data as numeric array from file
dem = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/dem/16m.tif')
Slope = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/Slope/16m.tif')
PlanCurvature = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/PlanCurvature/16m.tif')
RelativeTopographicPosition = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/RelativeTopographicPosition/16m.tif')
PennockLandFormClass = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/PennockLandFormClass/16m.tif')
TopographicWetnessIndex = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/TopographicWetnessIndex/16m.tif')
WILT = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/WILT/16m.tif')
DownslopeIndex_2m = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/DownslopeIndex_2m/16m.tif')
DownslopeIndex_4m = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/DownslopeIndex_4m/16m.tif')
DepthToWater_1ha = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/DepthToWater_1ha/16m.tif')
DepthToWater_2ha = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/DepthToWater_2ha/16m.tif')
DepthToWater_4ha = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/DepthToWater_4ha/16m.tif')
DepthToWater_8ha = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/DepthToWater_8ha/16m.tif')
DepthToWater_16ha = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/DepthToWater_16ha/16m.tif')
DepthToWater_32ha = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/DepthToWater_32ha/16m.tif')
ElevationAboveStream_1ha = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/ElevationAboveStream_1ha/16m.tif')
ElevationAboveStream_2ha = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/ElevationAboveStream_2ha/16m.tif')
ElevationAboveStream_4ha = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/ElevationAboveStream_4ha/16m.tif')
ElevationAboveStream_8ha = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/ElevationAboveStream_8ha/16m.tif')
ElevationAboveStream_16ha = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/ElevationAboveStream_16ha/16m.tif')
ElevationAboveStream_32ha = gdal_array.LoadFile('/content/Geographical-Intelligence/data/rasters/ElevationAboveStream_32ha/16m.tif')



Make a list of all arrays you wish to include. Note that you need to add or remove the variables in both the list and the converted dataframe.

In [ ]:
# Make a list of all arrays. you can 
list_or_all_rasters = [dem, Slope, PlanCurvature, RelativeTopographicPosition, PennockLandFormClass, TopographicWetnessIndex, WILT, DownslopeIndex_2m, DownslopeIndex_4m, DepthToWater_1ha, DepthToWater_2ha, DepthToWater_4ha, DepthToWater_8ha, DepthToWater_16ha, DepthToWater_32ha, ElevationAboveStream_1ha, ElevationAboveStream_2ha, ElevationAboveStream_4ha, ElevationAboveStream_8ha, ElevationAboveStream_16ha, ElevationAboveStream_32ha]

all_data = np.array(list_or_all_rasters)
all_data=all_data.reshape(21,738*662).T # The shape is from the original DEM

df_data=pd.DataFrame(all_data,columns=['dem', 'Slope', 'PlanCurvature', 'RelativeTopographicPosition', 'PennockLandFormClass', 'TopographicWetnessIndex', 'WILT', 'DownslopeIndex_2m', 'DownslopeIndex_4m', 'DepthToWater_1ha', 'DepthToWater_2ha', 'DepthToWater_4ha', 'DepthToWater_8ha', 'DepthToWater_16ha', 'DepthToWater_32ha', 'ElevationAboveStream_1ha', 'ElevationAboveStream_2ha', 'ElevationAboveStream_4ha', 'ElevationAboveStream_8ha', 'ElevationAboveStream_16ha', 'ElevationAboveStream_32ha'])


Now we can finally apply the prediction to the data. Once the prediction is done you can download the predicted raster and open it in your favorite GIS software such as Qgis. 

In [ ]:
result = rf_random.predict(df_data)

# Save the data as a raster file with coordinates and extent from one of the input layers
result = result.reshape(738,662)
extent = rasterio.open('/content/Geographical-Intelligence/data/rasters/dem/16m.tif')

with rasterio.Env():
  profile = extent.profile
  with rasterio.open('/content/Geographical-Intelligence/data/rasters/prediction.tif', 'w', **profile) as dst:
        dst.write(result, 1)


You can also plot the result for a quick inspection

In [ ]:
import rasterio
from matplotlib import pyplot
src = rasterio.open('/content/Geographical-Intelligence/data/rasters/prediction.tif')
pyplot.imshow(src.read(1), cmap='RdYlBu')
pyplot.show()

# Predict C/N ratio based on topographical data
---

Now you will do the same as above but instead of using classified soil moisture you will try to predict the C/N ratio from a new set of field plots from Krycklan. The data can be found here: /content/Geographical-Intelligence/data/Krycklan_Chemdata.csv

Remember to change from  [classification](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=randomforest#sklearn.ensemble.RandomForestClassifier) to [regression](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html?highlight=randomforest#sklearn.ensemble.RandomForestRegressor) and drop the apropriate columns before training the model.

Good luck!




In [ ]:
import pandas as pd
chemdata = pd.read_csv('/content/Geographical-Intelligence/data/Krycklan_Chemdata.csv', sep=';')
chemdata

# Extract more variables with Whitebox Tools
---
[Whitebox Tools](https://www.whiteboxgeo.com/manual/wbt_book/preface.html) Is a great software topographical modeling. This section describes how to extract additional topographical features. This is an example on how to set up Whitebox Tools and extract aspect from the original DEM.










In [ ]:
!pip install whitebox
import whitebox
wbt = whitebox.WhiteboxTools()

wbt.aspect(
    dem = '/content/Geographical-Intelligence/data/rasters/dem/16m.tif', 
    output = '/content/Geographical-Intelligence/data/rasters/aspect.tif', 
    zfactor=None
)

